In [1]:
import os
import pickle

In [2]:
with open('experiments/configs/small_datasets.pkl', 'rb') as f:
    config = pickle.load(f)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [4]:
from modules.logging.format_utils import format_measures
from modules.logging.logger import DefaultLogger
from modules.collecting.results_collector import DataFrameCollector
from modules.algorithms.nn.OSLPP_NN_UTILS import train_osda
from modules.algorithms.base.OSLPP import Params
from modules.selection.uncertanties import SelectRejectMode

results = DataFrameCollector({'source': [], 'target': [], 'desc': [], 'lr': [], 'seed': [], 'epochs': [], 'tops': []})
logger = DefaultLogger()
for (source, target), (common, tgt_private) in config.items():
    for epochs in [10]:
        for lr in [1e-3]:
            for n_r in [0.1, 0.25]:
#                 for weights in [[0.33, 0.33, 0.33], [0.2, 0.4, 0.4], [0.4, 0.2, 0.4], [0.4, 0.4, 0.2]]:
                 for tops in [[SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY],
                                [SelectRejectMode.CONFIDENCE, SelectRejectMode.MARGIN],
                                [SelectRejectMode.ENTROPY, SelectRejectMode.MARGIN],
                                [SelectRejectMode.CONFIDENCE, SelectRejectMode.ENTROPY, SelectRejectMode.MARGIN]]:
                    for seed in range(5):
                        params = Params(pca_dim=512, proj_dim=128, T=10, n_r=n_r, n_r_ratio=None,
                                    dataset='DomainNet_DCC', source=source, target=target,
                                    num_common=len(common), num_src_priv=0, num_tgt_priv=len(tgt_private))
                        metrics = train_osda(params, lr, epochs, SelectRejectMode.TOPS, seed, common, tgt_private, logger, tops=tops)
                        results.collect({'source': source, 'target': target, 'desc': format_measures(metrics), 'lr': lr, 'seed': seed, 'tops': tops, 'epochs': epochs})
results.get_result().to_csv('./all_results/results_weighted_scores/dcc__conf__small__NEW__combined_tops__nn_raw.csv', header=True, index=False)

painting -> sketch lr= 0.001 seed= 0
______
Iteration t=1
all:  cs/acc_i=51.66 cs/acc_c=53.15 os/recall_knw=90.30 os/recall_unk=27.55 total/acc_i=41.43 total/acc_c=49.82 total/h_score=35.76
selected:  cs/acc_i=58.84 cs/acc_c=60.24 os/recall_knw=74.75 os/recall_unk=74.25 total/acc_i=60.63 total/acc_c=55.60 total/h_score=62.97
______
Iteration t=2
all:  cs/acc_i=50.68 cs/acc_c=52.25 os/recall_knw=86.00 os/recall_unk=37.76 total/acc_i=43.73 total/acc_c=48.28 total/h_score=42.58
selected:  cs/acc_i=55.15 cs/acc_c=57.13 os/recall_knw=78.75 os/recall_unk=59.42 total/acc_i=53.26 total/acc_c=52.23 total/h_score=55.39
______
Iteration t=3
all:  cs/acc_i=50.87 cs/acc_c=52.40 os/recall_knw=84.94 os/recall_unk=39.59 total/acc_i=44.30 total/acc_c=48.18 total/h_score=43.64
selected:  cs/acc_i=53.91 cs/acc_c=55.81 os/recall_knw=82.15 os/recall_unk=47.38 total/acc_i=48.63 total/acc_c=51.15 total/h_score=49.28
______
Iteration t=4
all:  cs/acc_i=51.55 cs/acc_c=53.09 os/recall_knw=84.91 os/recall_unk=39

KeyboardInterrupt: 